In [1]:
import pickle as pkl
import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import copulagp.bvcopula as bvcopula
from copulagp.vine import CVine

i=1
with open(f'../../trained_models/ST260_Day1_Dataset_trained.pkl',"rb") as f:
    trained = pkl.load(f)

device = 'cuda:0'

# Figure 5G

In [ ]:
# calculate H(y,v|x)

N_points = 200
inputs = torch.linspace(0,1,N_points,device=device).float()

Hs = []

for reps in tqdm(range(10)):
    %time vine = CVine.sample_from_GP(trained['models'],inputs)
    sem_tol = 0.1
    %time H20 = vine.entropy(sem_tol=sem_tol,mc_size=2000,v=True).cpu().numpy()

    Hs.append([H20])

In [ ]:
for h in Hs:
    plt.plot(h[0])

In [ ]:
with open(f"../../plos_fig5_data/HYgX_109V_all.pkl","wb") as f:
    pkl.dump([20,inputs.cpu().numpy(),Hs],f)

# Figure 5H

In [2]:
Ns = [108,100,90,80,70,60,50,40,30,20,10,9,8,7,6,5,4,3,2]

In [4]:
# calculate I(u^x,x) with CopulaGP 'integrated'
MIs = []

assert len(trained['models'][0])==108

sem_tol = 0.1

reshuffle = np.arange(len(trained['X']))
np.random.shuffle(reshuffle)
inputs = torch.tensor(trained['X'][reshuffle[:1000]],device=device).float() # then inputMI takes s_mc_size subsamples again 

for i in tqdm(Ns):
    layers = [trained['models'][j][:i-j] for j in range(min(i-1,len(trained['models'])))]
    
    MI = []
    for j in range(3): # sample from a GP a few times
        %time vine = CVine.sample_from_GP(layers,inputs)
        %time mi = vine.inputMI(sem_tol=sem_tol, s_mc_size=50, r_mc_size=10, sR_mc_size=1000, v=True)
        MI.append(mi)
        
    print(np.mean(MI))

    MIs.append([i,MI])
    
with open("../../plos_fig5_data/MI_CopulaGP_integrated.pkl","wb") as f:
    pkl.dump([Ns,MIs],f)

CPU times: user 34 s, sys: 376 ms, total: 34.4 s
Wall time: 57.5 s
Start calculating p(r) 0
Finished in 4 steps
19.4,16.4,2.99,                        11.4,8.35
Start calculating p(r) 1
Finished in 5 steps
19.2,16.2,2.98,                        8.26,5.88
Start calculating p(r) 2
Finished in 7 steps
18.4,15.7,2.68,                        6.12,4.6
Start calculating p(r) 3
Finished in 4 steps
18.3,15.7,2.62,                        5.13,3.97
Start calculating p(r) 4
Finished in 3 steps
18.3,15.8,2.55,                        4.46,3.58
Start calculating p(r) 5
Finished in 3 steps
18.3,15.7,2.57,                        4.08,3.26
Start calculating p(r) 6
Finished in 4 steps
18.4,15.8,2.58,                        3.8,3.07
Start calculating p(r) 7
Finished in 3 steps
18.5,15.9,2.6,                        3.57,2.91
Start calculating p(r) 8
Finished in 4 steps
18.4,15.8,2.58,                        3.34,2.74
Start calculating p(r) 9
Finished in 3 steps
18.2,15.7,2.55,                        3.12,2

Start calculating p(r) 36
Finished in 4 steps
18.0,15.5,2.48,                        1.59,1.33
Start calculating p(r) 37
Finished in 5 steps
18.0,15.5,2.48,                        1.57,1.31
Start calculating p(r) 38
Finished in 5 steps
18.0,15.5,2.48,                        1.55,1.3
Start calculating p(r) 39
Finished in 2 steps
18.1,15.6,2.48,                        1.53,1.28
Start calculating p(r) 40
Finished in 4 steps
18.1,15.6,2.48,                        1.51,1.27
Start calculating p(r) 41
Finished in 5 steps
18.0,15.6,2.48,                        1.49,1.25
Start calculating p(r) 42
Finished in 4 steps
18.0,15.6,2.48,                        1.47,1.23
Start calculating p(r) 43
Finished in 2 steps
18.0,15.5,2.49,                        1.46,1.22
Start calculating p(r) 44
Finished in 4 steps
18.0,15.5,2.49,                        1.44,1.21
Start calculating p(r) 45
Finished in 3 steps
18.1,15.6,2.49,                        1.43,1.19
Start calculating p(r) 46
Finished in 3 steps
18.1,

Finished in 4 steps
18.4,15.8,2.58,                        2.24,1.85
Start calculating p(r) 20
Finished in 5 steps
18.4,15.8,2.6,                        2.18,1.8
Start calculating p(r) 21
Finished in 3 steps
18.4,15.8,2.59,                        2.13,1.77
Start calculating p(r) 22
Finished in 3 steps
18.4,15.8,2.58,                        2.08,1.73
Start calculating p(r) 23
Finished in 6 steps
18.3,15.7,2.58,                        2.03,1.69
Start calculating p(r) 24
Finished in 4 steps
18.3,15.7,2.57,                        1.98,1.65
Start calculating p(r) 25
Finished in 5 steps
18.4,15.8,2.59,                        1.96,1.62
Start calculating p(r) 26
Finished in 3 steps
18.3,15.7,2.59,                        1.93,1.59
Start calculating p(r) 27
Finished in 2 steps
18.4,15.8,2.6,                        1.9,1.57
Start calculating p(r) 28
Finished in 3 steps
18.5,15.8,2.61,                        1.88,1.55
Start calculating p(r) 29
Finished in 6 steps
18.4,15.8,2.61,                   

AttributeError: 'torch.dtype' object has no attribute 'type'

In [ ]:
# calculate I(u^x,x) with CopulaGP 'estimated'

with open(f'../../trained_models/ST260_Day1_DatasetS_trained.pkl',"rb") as f:
    trainedS = pkl.load(f)
    
L = min(len(trained['models']),len(trainedS['models']))
print(f"Taking {L} trees for both vines")
    
Npoints=200
reshuffle = np.arange(len(trained['X']))
np.random.shuffle(reshuffle)
inputs = torch.tensor(trained['X'][reshuffle[:Npoints]],device=device).float()

MIs_est = []
    
sem_tol = 0.1
    
for i in tqdm(Ns):
    layers = [trained['models'][j][:i-j] for j in range(min(i-1,L))]
    %time vine = CVine.marginalize(layers,inputs)
    %time H = vine.entropy(sem_tol=sem_tol,mc_size=2000,v=True).cpu().numpy()
    
    layers = [trainedS['models'][j][:i-j] for j in range(min(i-1,L))]
    %time vineS = CVine.marginalize(layers,inputs)
    %time HS = vineS.entropy(sem_tol=sem_tol,mc_size=2000,v=True).cpu().numpy()
    
    print(np.mean(HS-H))

    MIs_est.append([i,np.mean(HS)-np.mean(H)])
    
with open("../../plos_fig5_data/MI_CopulaGP_estimated.pkl","wb") as f:
    pkl.dump([Ns,MIs_est],f)

Taking 59 trees for both vines


CPU times: user 37 s, sys: 97.4 ms, total: 37.1 s
Wall time: 1min 16s
tensor(2.1682, device='cuda:0')
tensor(1.5422, device='cuda:0')
tensor(1.2428, device='cuda:0')
tensor(1.0802, device='cuda:0')
CPU times: user 8min, sys: 839 ms, total: 8min 1s
Wall time: 13min 2s
CPU times: user 36.9 s, sys: 214 ms, total: 37.1 s
Wall time: 1min 12s
tensor(1.3183, device='cuda:0')
CPU times: user 2min 41s, sys: 726 ms, total: 2min 42s
Wall time: 2min 46s
3.4111876
CPU times: user 31.9 s, sys: 220 ms, total: 32.2 s
Wall time: 33.5 s
tensor(2.2405, device='cuda:0')
tensor(1.5950, device='cuda:0')
tensor(1.2971, device='cuda:0')
tensor(1.1242, device='cuda:0')
tensor(1.0053, device='cuda:0')
CPU times: user 8min 10s, sys: 2.7 s, total: 8min 12s
Wall time: 7min 18s
CPU times: user 31.8 s, sys: 455 ms, total: 32.2 s
Wall time: 33.2 s
tensor(1.3288, device='cuda:0')
CPU times: user 2min 25s, sys: 759 ms, total: 2min 26s
Wall time: 1min 51s
3.3480606
CPU times: user 26.9 s, sys: 413 ms, total: 27.3 s
Wall

In [8]:
# calculate I(u^x,x) with KSG

from copulagp.MI import BI_KSG

MI_KSG = []
for i in tqdm.tqdm(Ns):
    mi, _ = BI_KSG(trained['X'].reshape(-1,1),trained['Y'][:,:i])
    MI_KSG.append(mi)
    print(mi)
with open("../../plos_fig5_data/MI_KSG.pkl","wb") as f:
    pkl.dump([Ns,MI_KSG],f)

-0.06099654895066115
-0.03986131268375232
-0.02927459997843107
-0.02200298898868935
-0.01611234885982857
0.01112668345155461
0.028912592706137132
0.05754269508284846
0.08710677208899811
0.08894959467831991
0.086226130193683
0.09026774374123746
0.07954997377914003
0.10223551203960383
0.10685752234427694
0.09784357758419601
0.10447502863036043
0.1096893255932253
0.1020234450043036



In [7]:
# calculate I(u^x,x) with MINE

from copulagp.MI import train_MINE

MI_MINE = []
for i in tqdm.tqdm(Ns):
    mi = train_MINE(trained['Y'][:,:i], x = torch.tensor(trained['X'],device=device).float(),H=500,device=device)
    MI_MINE.append(mi)
with open("../../plos_fig5_data/MI_MINE.pkl","wb") as f:
    pkl.dump([Ns,MI_MINE],f)